#### Two-segments model

In [ ]:
import numpy as np
import McNeuron
import data_transforms

from keras.models import Sequential
from keras.layers.core import Dense, Reshape
from keras.layers.recurrent import LSTM

import matplotlib.pyplot as plt
from copy import deepcopy
import os

from numpy import linalg as LA
import data_transforms
%matplotlib inline
from keras import backend as K

l = 50000
subsampling_numbers = 20
morph = np.zeros([l, subsampling_numbers - 1])
geo = np.zeros([l, subsampling_numbers - 1, 3])
data = dict()
for i in range(l):
    k = np.floor((subsampling_numbers-2)*np.random.rand())
    k = np.array(k, dtype=int)
    k = 4
    n = np.floor(k*np.random.rand())
    n = np.array(n, dtype=int)
    par = np.append(np.arange(0,k+1),n)
    par = np.append(par,np.arange(k+2,subsampling_numbers-1))
    par = np.append(-1, par)

    #morph[i, :] = data_transforms.encode_prufer(par.tolist())
    morph[i, :] = par[1:]
    geo[i, 0, 0:2] = np.random.rand(2)
    for j in range(1,k+1):
        #geo[i, j, 0:2] =  geo[i, j-1, 0:2] + np.random.rand(2)
        geo[i, j, 0:2] = np.random.rand(2)
    a = np.random.rand(2)
    a[0] = -a[0]
    #geo[i ,k+1 ,0:2] = geo[i, n-1, 0:2] + a
    geo[i ,k+1 ,0:2] = a
    
    for j in range(k+2,subsampling_numbers-1):
        a = np.random.rand(2)
        a[0] = -a[0]
        #geo[i, j, 0:2] = geo[i, j-1, 0:2] + a
        geo[i, j, 0:2] = a

data['morphology'] = dict()
data['morphology']['n'+str(subsampling_numbers)] = morph
data['geometry'] = dict()
data['geometry']['n'+str(subsampling_numbers)] = geo

# Visulazing

In [ ]:
M = np.zeros([subsampling_numbers-1,7])
M[1:,6] = morph[0,:-1]+1
M[0,6] = -1
M[np.ix_(range(1,subsampling_numbers-1),range(2,5))] = np.squeeze(geo[0,:-1,:])
n = McNeuron.Neuron(file_format = 'Matrix of swc', input_file = M)
n.distance_from_parent = np.append(0,geo)
n.set_connection()
x = n.connection
x[np.isnan(x)] = 0
#n.location = np.dot(x, np.append([[0,0,0]],np.squeeze(geo), axis = 0)).T
n.location = np.dot(x, np.squeeze(geo)).T
McNeuron.visualize.plot_2D(n, save="e.eps")
#n.fit()
#McNeuron.visualize.plot_dedrite_tree(n)
#print n.parent_index
#print par
#print n.location

#### random_attached Model

In [ ]:
def random_unitary_basis( kappa):
    Ax1 = random_2d_rotation_in_3d('x', kappa)
    Ay1 = random_2d_rotation_in_3d('y', kappa)
    Az1 = random_2d_rotation_in_3d('z', kappa)
    Ax2 = random_2d_rotation_in_3d('x', kappa)
    Ay1 = random_2d_rotation_in_3d('y', kappa)
    Az1 = random_2d_rotation_in_3d('z', kappa)
    A = np.dot(np.dot(Ax1,Ay1),Az1)
    B = np.dot(np.dot(Az1,Ay1),Ax1)
    return np.dot(A,B)

def random_2d_rotation_in_3d(axis, kappa):
    theta = np.random.vonmises(0, kappa, 1)
    A = np.eye(3)
    if axis is 'z':
        A[0,0] = np.cos(theta)
        A[1,0] = np.sin(theta)
        A[0,1] = - np.sin(theta)
        A[1,1] = np.cos(theta)
        return A
    if axis is 'y':
        A[0,0] = np.cos(theta)
        A[2,0] = np.sin(theta)
        A[0,2] = - np.sin(theta)
        A[2,2] = np.cos(theta)
        return A
    if axis is 'x':
        A[1,1] = np.cos(theta)
        A[2,1] = np.sin(theta)
        A[1,2] = - np.sin(theta)
        A[2,2] = np.cos(theta)
        return A
    
l = 1
subsampling_numbers = 20
morph = np.zeros([l, subsampling_numbers])
geo = np.zeros([l, subsampling_numbers - 1, 3])
data = dict()
for i in range(l):
    par 
    b = 3
    B = np.array([2])
    ind = 0
    leaves = np.array([0])
    while(ind<subsampling_numbers-1):
        index = int(np.floor(np.random.rand()*len(leaves)))
        a = leaves[index]
        if(ind>0):
            (parent,) = np.where(morph[i, :] == morph[i, a])
            parent = parent[0]
        else:
            parent = 0
        # if we want to make a branch
        if(np.any(B==ind)):
            s = geo[i, a,:]
            pert = 0.1*np.random.randn(3)
            l_par = geo[i,parent,:] - geo[i,a,:]
            geo[i,ind,:] = s + np.dot(random_unitary_basis(20), l_par) + pert
            geo[i,ind+1,:] = s + np.dot(random_unitary_basis(20), l_par) - pert
            leaves = np.append(leaves, ind)
            leaves = np.append(leaves, ind+1)
            leaves = np.delete(leaves, index)
            morph[i, ind] = a
            morph[i, ind+1] = a
            ind = ind + 2
            
        # if we want to extend a segment   
        else:
            s = geo[i,a,:]
            pert = 0.1*np.random.randn(1,3)
            l_par = geo[i,parent,:] - geo[i,a,:]
            geo[i,ind,:] = s + np.dot(random_unitary_basis(20), l_par) + pert
            leaves = np.append(leaves, ind)
            leaves = np.delete(leaves, index)
            morph[i, ind] = a
            ind = ind + 1
            
    #par = np.append(-1, par)
    par[0] = -1

data['morphology'] = dict()
data['morphology']['n'+str(subsampling_numbers)] = morph[:,:-1]
data['geometry'] = dict()
data['geometry']['n'+str(subsampling_numbers)] = geo[:,:-1,:]

In [ ]:
random_unitary_basis(20)

In [ ]:
morph[0, :-1]

In [ ]:
M = np.zeros([subsampling_numbers-1,7])
M[:,6] = morph[0,:-1]+1
M[0,6] = -1
M[np.ix_(range(1,subsampling_numbers-1),range(2,5))] = np.squeeze(geo[0,:-1,:])
n = McNeuron.Neuron(file_format = 'Matrix of swc', input_file = M)
#n.distance_from_parent = np.append(0,geo)
n.set_connection()
#x = n.connection
#x[np.isnan(x)] = 0
#n.location = np.dot(x, np.append([[0,0,0]],np.squeeze(geo), axis = 0)).T
McNeuron.visualize.plot_2D(n)
#n.fit()
#McNeuron.visualize.plot_dedrite_tree(n)
print n.parent_index
#print par
print n.location

In [ ]:
McNeuron.visualize.plot_dedrite_tree(n)

In [ ]:
n.parent_index

In [ ]:
print n.location.shape

In [ ]:
n.distance_from_parent = geo
n.set_connection
n.location = np.dot(n.connection, np.append([[0,0,0]],np.squeeze(geo), axis = 0))

In [ ]:
import pickle
pickle.dump(data, open( "synthetic_Two_segment_model_Different_directions_Parent_distance_n20_parent_id_loc=dis_parent_k4.p", "wb" ))

In [ ]:
import data_transforms
data_transforms.make_swc_from_prufer_and_locations(data)

In [ ]:
import plotly.plotly as py
py.sign_in('RoozbehFarhoodi', '0ndQhPE4QGdQ39JBdFPg')
import plotly.graph_objs as go

import numpy as np

x, y, z = np.random.multivariate_normal(np.array([0,0,0]), np.eye(3), 20).transpose()
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)

x2, y2, z2 = np.random.multivariate_normal(np.array([0,0,0]), np.eye(3), 20).transpose()
trace2 = go.Scatter3d(
    x=x2,
    y=y2,
    z=z2,
    mode='markers',
    marker=dict(
        color='rgb(127, 127, 127)',
        size=12,
        symbol='circle',
        line=dict(
            color='rgb(204, 204, 204)',
            width=1
        ),
        opacity=0.9
    )
)
data = [trace1, trace2]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')